In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import os
import gc
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
def reset_cuda():
    """Complete CUDA reset"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        gc.collect()
        
        # Reset CUDA context (this is the key part)
        try:
            torch.cuda.reset_peak_memory_stats()
            print("✓ CUDA memory stats reset")
        except:
            pass
            
        # Try to clear any existing CUDA context
# Perform reset
reset_cuda()


print("=== CUDA DIAGNOSTIC ===")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    # Test basic CUDA operations
    try:
        x = torch.randn(2, 2, device='cuda')
        y = torch.randn(2, 2, device='cuda')
        z = x + y
        print("✓ Basic CUDA tensor operations work")
    except Exception as e:
        print(f"❌ Basic CUDA operations failed: {e}")
        print("Your GPU/CUDA installation has issues")



PyTorch Version: 2.7.1+cu128
CUDA Available: True
Using device: cuda
✓ CUDA memory stats reset
=== CUDA DIAGNOSTIC ===
PyTorch version: 2.7.1+cu128
CUDA available: True
CUDA version: 12.8
GPU device: NVIDIA GeForce MX330
GPU memory: 2.1 GB
✓ Basic CUDA tensor operations work


In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
text[:100]
vocab = sorted(list(set(text)))
vocab_size = len(vocab)

In [4]:
"""def encode(x):
    stoi = []
    for i in x:
        stoi.append(vocab.index(i))
    return stoi

def decode(x):
    itos = ""
    for i in x:
        itos = itos + (vocab[int(i)])
    return itos

"""


stoi = { ch:i for i,ch in enumerate(vocab) }
itos = { i:ch for i,ch in enumerate(vocab) }
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l]) 


In [15]:
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 128 # what is the maximum context length for predictions?
max_iters = 5000
eval_iters = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
n_embd = 256
n_head = 4
head_size = n_embd//n_head
n_layer = 4
dropout = 0.2


device = torch.device("cuda")
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    
    X = torch.stack([data[i:i+block_size] for i in ix])
    Y = torch.stack([data[i+1:i+1+block_size] for i in ix])

    
    # Only transfer to GPU if values look valid
    if hasattr(torch.cuda, 'is_available') and torch.cuda.is_available():
        X, Y = X.to(device), Y.to(device)
    
    return X, Y

In [16]:
class AttentionHead(nn.Module):
    def __init__(self , head_size):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size, bias= False)
        self.key = nn.Linear(n_embd, head_size, bias= False)
        self.value = nn.Linear(n_embd, head_size, bias= False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)   #B, T, hs
        v = self.value(x)
        wei = q @ k.transpose(-2, -1) / ((k.shape[-1])**0.5)
        mask = self.tril[:T, :T]
        wei = wei.masked_fill(mask == 0, float('-inf'))
        wei = F.softmax(wei, dim= -1)        
        out  = wei @ v
        out = self.dropout(out)
        return out
    


In [17]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList(AttentionHead(head_size) for _ in range(n_head))
        self.proj = nn.Linear(n_head * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([heads(x) for heads in self.heads], dim = -1)
        out = self.dropout(self.proj(out))
        return out

class Feedforward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)
    

class Block(nn.Module):
    def __init__(self, n_head, head_size):
        super().__init__()
        self.sa = MultiHeadedAttention(n_head, head_size)
        self.ffwd = Feedforward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

        


In [18]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, n_embd):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.pos_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_head, head_size) for _ in range(n_layer)])
        self.fln = nn.LayerNorm(n_embd)
        self.final = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, x, targets = None):
        B, T = x.shape
        
        # Token embeddings
        tok_emb = self.token_embedding_table(x)  # (B, T, n_embd)
        
        # Position embeddings - create position indices from 0 to T-1
        pos_idx = torch.arange(T, device=x.device)  # (T,)
        pos_emb = self.pos_embedding_table(pos_idx)  # (T, n_embd)
        
        # Combine token and position embeddings
        x = tok_emb + pos_emb  # Broadcasting: (B, T, n_embd) + (T, n_embd)
        
        x = self.blocks(x)
        x = self.fln(x)
        logits = self.final(x)
        
        if targets == None:
            loss = None
            
        else:
            B, T, C = logits.shape
            
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_context_length=500):
        for _ in range(max_context_length):
            # Only use last block_size tokens
            idx_cond = idx if idx.size(1) <= block_size else idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
        return idx
                
            


In [19]:
model = Transformer(vocab_size, n_embd)
m = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

3.222593 M parameters


In [20]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_iters == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()




step 0: train loss 4.2160, val loss 4.2138
step 500: train loss 1.9728, val loss 2.0425
step 1000: train loss 1.6436, val loss 1.8119
step 1500: train loss 1.5180, val loss 1.7063
step 2000: train loss 1.4479, val loss 1.6494
step 2500: train loss 1.4010, val loss 1.6256
step 3000: train loss 1.3690, val loss 1.5768
step 3500: train loss 1.3445, val loss 1.5726
step 4000: train loss 1.3205, val loss 1.5543
step 4500: train loss 1.2973, val loss 1.5381
step 4999: train loss 1.2766, val loss 1.5285


In [21]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, 500)[0].tolist()))


Of I heart; I ready our first will depart?
O countroft us fallow of a loving of thine;
Go, again this slaint, thinkings may capital
Over hop enemied; poor profess in many fretch
A beg olden? and my fortune for, I pray beg thine.
I'll rathe patient ooh time which buy doth
not plent since.

First Withinous 'last thus of untendance to ground?

DUKE VIN OF SAYRREEL:
My lady mother depart to betice
That God, sight a set loving the Coruncre,
Will be rise and lenpet the earth.
Unless truth obiling a be


In [22]:
torch.save(model.state_dict(), 'model_weights.pth')